# TP2: Seafloor classification par CNN 

## 1 - Introduction

Disposant d'un ensemble d'images dont on veut prédire la classe, deux possibilités s'offrent pour apprendre un modèle profond de classement.

- La première possibilité se nomme "Transfer Learning" associé au "fine tuning" dont les principes sont d'utiliser un réseau de neurones profond entrainé dans un autre contexte et de l'adapter à nos données: 
    
- La seconde possibilité est de créer et d'entrainer un modèle profond ex-nihilo (from scratch, en partant de zéro).


L'objectif de ce TP est d'appliquer ces deux possibilités au problème de classification de patchs d'images sonar en types de fond marin que vous avez déjà traités dans les TPs précédent. Vous reprendrez les fonctions d'import des patchs que vous avez déjà mises au point lors des tps précédents.

Il pourra être néanmoins utile d'utiliser les lignes de code suivantes pour que les données soient dans la forme attendue par tensoflow/keras et éventuellement changer la taille des images:
```python
from keras.preprocessing.image import img_to_array, load_img
target_size = 192;
feature_values = np.array([img_to_array(load_img(img,  # color_mode = "grayscale",
             target_size=(target_size, target_size))) for img in dataset_df['image_path'].values.tolist()
]).astype('float32')
```
A noter que la taille des patchs est automatiquement réduite à 192x192 pixels, vous pouvez réduire cette taille en fonction des performances de votre machine. Attention, une taille trop basse peut impliquer une erreur si vous réutilisez par la suite un modèle déjà entrainé (par exemple VGG16).

La création d'ensemble d'apprentissage, de validation et de test se fera en divisant la base en trois parts. Il faudrait pour ce petit jeu de données réaliser une procédure de cross-validation. Compte tenu du temps pour réaliser cette procédure, elle sera ici laissée de coté. 

## 1 - Tutorials CNN et transfer learning par fine tuning

Dans le startercode, vous trouverez un jupyter notebooks qui vous servira de base pour réaliser la suite. Dans un premier temps, le notebook détaille le fonctionnement et la mise en oeuvre des CNNs (en particulier les différentes couches d'un CNN avec des exemples); pour ensuite détailler la procédure liée au fine tuning avec data augmentation à partir de données de type TensorFlow Dataset. 

Veillez à bien suivre les différentes étapes et à bien comprendre les différentes commandes employées. Si vous voulez d'autres exemples, vous trouverez d'autres ressources supplémentaires.


## 2 - Transfer learning par fine tuning sur le dataset seafloor

- Vous commencerez par le fine tuning en vous inspirant du tuto fourni ci-dessus pour faire du transfer learning du modèle xception (dont les paramètres ont été appris sur la base d'images "imageNet") pour l'appliquer aux patchs d'images sonar. Vous procéderez ainsi:
  - les modèles sont téléchargeables ici si vous avez des problèmes pour télécharger: https://drive.google.com/open?id=1qFwqoNU1fsvl8fu-7eRCmjoPNZZNzPSJ
  - Résumer l'approche du transfer learning/fine tuning
  - Décriver l'architecture du modèle utilisé (xception ici)
  - Vous précisérez votre choix concernant les paramètres des fonctions appelées en particulier expliquer votre démarche concernant les phases de preprocessing des images, de data augmentation, de classification, etc.
  - remarque: comme les images sonar sont en niveaux de gris et que le modèle VGG prend en entrée des images couleurs, il s'agira de dupliquer ce canal sur les canaux R, G et B. 

  - Enfin, vous évaluerez les performances obtenues.
- (Bonus) comparez les résultats obtenus par l'architecture vgg16 (https://keras.io/applications/).

- (Bonus) Essayez et comparez les résultats obtenus par d'autres architectures (Resnet, Inception etc...https://keras.io/applications/).

## 3 - Proposition de votre propre achitecture  

- Vous proposerez ensuite une architecture de réseau profond convolutif et évaluerez ses performances. 
- Expliquez brièvement votre architecture et en particulier à quoi servent les couches (et leur enchainement) de votre architecture.
- Vous comparerez ensuite les performances obtenus (par rapport à ceux obtenus à la partie précédente) sur la matrice de confusion et les métriques de performance classiques.






# A rendre 
- pour le **12/01/21**
- la séance du 05/01/21 sera consacrée à finaliser
- **Commenter au maximum votre code (pourquoi vous utilisez tel ou tel bout de code) ou apporter des précisions dans votre CR.**
- au choix (**N'oublier pas les deux noms en cas de binômes**):
    - un fichier zip avec *.py et un cr au format pdf
    - un fichier .ipynb avec compte-rendu et code



# 4 - Aide pour démarrer

## 4.1 Chargement des données
Vous pourrez utiliser cette procédure pour charger les données: 
**A noter**
- target_size permet de définir un éventuel changement de taille des images qui pourra servir en fonction de la taille d'entrée du modèle que vous considérez.
- comme les images sonar sont en niveaux de gris et que le modèle VGG prend en entrée des images couleurs, load_img duplique ce canal sur les canaux R, G et B.

In [1]:
import tensorflow.keras as keras
import pandas as pd
import os 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from sklearn import preprocessing


import sys
IN_COLAB = 'google.colab' in sys.modules

# Paramètres
if IN_COLAB:
    DATASET_PATH = 'gdrive/My Drive/Colab Notebooks/ex06_supervised_seabedClassification/dataset/imgs/'
    LABEL_PATH = 'gdrive/My Drive/Colab Notebooks/ex06_supervised_seabedClassification/dataset/labels/labels.csv'
else:
    IN_COLAB = False
    DATASET_PATH = r'./dataset/imgs/'
    LABEL_PATH = r'./dataset/labels/labels.csv'

    
# 
target_size = 200
    
    
# Charger le fichier CSV

dataset_df = pd.read_csv(LABEL_PATH)


# We add another column to the labels dataset to identify image path
dataset_df['image_path'] = dataset_df.apply(lambda row: (DATASET_PATH + row["id"]), axis=1)

batch_imgs = np.array([img_to_array(
    load_img(img,  # color_mode = "grayscale",
             target_size=(target_size, target_size))
) for img
    in dataset_df['image_path'].values.tolist()
]).astype('float32')


#  Noms des labels dans l'ordre, respectivement aux indices
labelNames_unique = np.array(["Posidonia","Ripple 45°","Ripple vertical","Rock","Sand","Silt"])
labelDict={}
for i in range(len(labelNames_unique)):
    labelDict.update({i:labelNames_unique[i]})

    
# Récupération des labels
label_names = dataset_df['seafloor']


# nb de classes
label_nb = labelNames_unique.shape[0]

# indices
le = preprocessing.LabelEncoder()
le.fit(labelNames_unique)
labelIndices_unique = le.transform(labelNames_unique)
labelIndices  = le.transform(label_names)


# one-hot-encoding
labelOhe = pd.get_dummies(label_names.reset_index(drop=True)).values



## 4.2 - Définition, entrainer et évaleur le modèle VGG16 (par exemple) pour le fine tuning
Ensuite procéder comme le tuto:
- charger le modèle VGG16 sans le classifieur include_top=False
- Visualiser l'architecture du modèle: model.summary()
- Créer un modèle d'extraction d'information (features) allant de la couche d'entrée de VGG16 jusqu'à sa dernière couche de convolution nommée 'block3_pool'
- Rajouter des couches Dense pour définir un classifieur Fully connected (attention aux nombres de sorties de la dernière couche et à sa fonction d'activation)
- ne pas oublier l'étape de preprocessing essentiel à la bonne réussite de l'apprentissage. Il faut se renseigner sur les corrections à apporter et la taille des images d'entrée
- Compiler (compile), entrainer (fit) et évaluer (evaluate) le modèle


## 4.3 - Fonctions d'aide


In [2]:
# fonction d'aide pour afficher les courbes d'apprentissage

def learningCurves(history,title):
    #Learning curve plotting
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    t = f.suptitle(title, fontsize=12)
    f.subplots_adjust(top=0.85, wspace=0.3)

    epochs = list(range(1,NB_EPOCHS+1))
    ax1.plot(epochs, history['accuracy'], label='Train Accuracy')
    ax1.plot(epochs, history['val_accuracy'], label='Validation Accuracy')
    ax1.set_xticks(epochs)
    ax1.set_ylabel('Accuracy Value')
    ax1.set_xlabel('Epoch')
    ax1.set_title('Accuracy')
    ax1.legend()

    ax2.plot(epochs, history['loss'], label='Train Loss')
    ax2.plot(epochs, history['val_loss'], label='Validation Loss')
    ax2.set_xticks(epochs)
    ax2.set_ylabel('Loss Value')
    ax2.set_xlabel('Epoch')
    ax2.set_title('Loss')
    ax2.legend()

    
# fonction callback (à rajouter en option à model.fit) pour suivre l'évolution de la matrice de confusion au long de l'apprentissage
# credit: K. Bedin (ROB 2020) développé lors du cours
class ConfusionEvaluation(Callback):
    """
        Fonction callback pour la méthode 'fit_generator()' permettant d'afficher 
        la matrice de confusion à chaque fin d'Epoch.
        Cela permet de visualiser concraitement l'évolution de la classification au cours de l'apprentissage.
    """
    def __init__(self, validation_data=()):
        super(Callback, self).__init__()
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        preds_Inception = self.model.predict(self.X_val/255.)
        matrixInception = confusion_matrix(self.y_val,preds_Inception.argmax(axis=1))
        print("\nConfusion Matrix:")
        print(matrixInception)

#cbk_matconf = ConfusionEvaluation(validation_data=(featValues_val, labelInd_val))

    

NameError: name 'Callback' is not defined

# 5 -  Ressources supplémentaires en transfer learning par fine tuning**
 En dehors des supports de cours, vous pourrez aussi vous appuyer sur:
- Les concepts du transfer learning sont expliqués dans les liens ci-dessous:
  - https://www.youtube.com/watch?v=FQM13HkEfBk&index=20&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF
  - http://cs231n.github.io/transfer-learning/
  - https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html et https://flyyufelix.github.io/2016/10/08/fine-tuning-in-keras-part2.html
- Des exemples supplémentaires d'implémentation
  - https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/CIFAR10_CNN_Classifier.ipynb et https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/CIFAR10_VGG16_Transfer_Learning_Classifier.ipynb
  - https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/Dog_Breed_EDA.ipynb et https://github.com/dipanjanS/hands-on-transfer-learning-with-python/blob/master/notebooks/Ch06%20-%20Image%20Recognition%20and%20Classification/Dog_Breed_Transfer_Learning_Classifier.ipynb
- cours et des vidéos de Stanford University: https://www.youtube.com/watch?v=wEoyxE0GP2M&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=6, https://www.youtube.com/watch?v=wEoyxE0GP2M&list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv&index=7)
 